# Собираем датасет

## Установка нужных библиотек

In [1]:
!pip install wandb
!pip install stable_baselines3
!pip install shimmy
!pip install gymnasium stable-baselines3
!pip install gym

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cu

## Вход в учетную запись Weights&Biases

In [2]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##Импорт нужных модулей


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import json
import pickle
import gym
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback
import wandb
from stable_baselines3 import A2C
import envs

/usr/local/lib/python3.10/dist-packages/wandb/analytics/sentry.py:90: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  self.hub = sentry_sdk.Hub(client)


## Теперь нужно описать, как мы будем сохранять траектории

In [4]:
class SaveTrajCallBack(BaseCallback):
    """
    Класс для сохранения и логирования траекторий
    """
    def __init__(self, verbose=0, env=None, lifetime_idx=None, log=False, log_interval=1000):
        """
        Инициализация callback

        :param verbose: Уровень verbose
        :param env: Среда, в которой происходит обучение
        :param lifetime_idx: Индекс жизненного цикла
        :param log: Флаг, указывающий, нужно ли логировать данные с использованием wandb
        :param log_interval: Интервал для логирования данных
        """
        super().__init__(verbose)
        self.env = env
        self.lifetime_idx = lifetime_idx
        self.log = log
        self.log_interval = log_interval

        self.trajectories = []

    def _on_step(self) -> bool:
        """
        Метод, вызываемый на каждом шаге обучения. Собирает текущие состояние, действие и награду,
        а также time step и добавляет в список траекторий.

        :return: True, если обучение должно продолжаться.
        """
        episode_length = self.env.episode_length
        learning_step = self.num_timesteps - 1

        state = self.locals.get('obs_tensor')[0].tolist()
        action = self.locals.get('actions')[0].tolist()
        reward = self.locals.get('rewards')[0].tolist()

        timestep = learning_step % episode_length

        self.trajectories.append((state, action, reward, timestep))

        log_interval = self.log_interval
        if (learning_step + 1) % log_interval == 0:
            mean_reward = np.mean([t[2] for t in self.trajectories[-log_interval:]])
            print(f'Шаг времени: {self.num_timesteps}, Средняя награда: {mean_reward}')

            if self.log:
                wandb.log({'mean_reward': mean_reward})

        return True

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
def collect_lifetimes(
        env_id: str,
        lifetime_num: int = 10,
        lifetime_start_idx: int = 0,
        total_steps: int = 10000,
        output_prefix: str = None,
        alg_config: str = None,
        log: bool = False,
) -> None:
    """
    Собирает lifetimes (жизненные циклы), используя алгоритм A2C

    Args:
        env_id: ID используемой среды Gym
        lifetime_num: Количество жизненных циклов, которые нужно сохранить
        total_steps: Общее количество шагов обучения

    Returns:
        Список жизненных циклов, каждый из которйо сконкатенирован в лист траекторий:
        [(o0^0, a0^0, r0^0, t0^0), (o1^0, a1^0, r1^0, t1^0), ..., , (oT^0, aT^0, rT^0, tT^0),
         (o0^1, a0^1, r0^1, t0^1), (o1^1, a1^1, r1^1, t1^1), ..., (oT^1, aT^1, rT^1, t1^1),
         ...]
        _t --  timestep, ^i -- индекс траектории
    """
    agent_config = json.load(open(alg_config, 'r'))

    if log:
        wandb.init(
            project='alg-distill',
            name=env_id + '-collect',
            monitor_gym=True,
            config={
                'env_id': env_id,
                'lifetime_num': lifetime_num,
                'total_steps': total_steps,
                **agent_config,
            }
        )

    for lifetime_idx in range(lifetime_start_idx, lifetime_num):
        print(f'Запуск lifetime {lifetime_idx}...')

        env = gym.make(env_id)

        alg = A2C(env=env, **agent_config)
        callback = SaveTrajCallBack(env=env, lifetime_idx=lifetime_idx, log=log)

        alg.learn(total_timesteps=total_steps, callback=callback)

        if output_prefix:
            pickle.dump(callback.trajectories, open(f"{output_prefix}_{lifetime_idx}.pkl", 'wb'))

###Функция для нахождения конфига DarkRoom

In [6]:
def find_config_file(env_id: str, alg: str):
    if env_id.startswith('DarkRoom'):
        return f"configs/DarkRoom-{alg}.json"
    else:
        raise ValueError(f"Неизвестная среда или алгоритм: {env_id, alg}")

## Собираем lifetimes

In [14]:
alg_config = find_config_file('DarkRoom-v0', 'a2c')

print(f'alg_config: {alg_config}')

collect_lifetimes(
    env_id='DarkRoom-v0',
    lifetime_num=2000,
    lifetime_start_idx=1800,
    total_steps=20000,
    output_prefix='darkroom_normal',
    alg_config=alg_config,
    log=True,
)

alg_config: configs/DarkRoom-a2c.json


Выходные данные были обрезаны до нескольких последних строк (5000).
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 20       |
|    ep_rew_mean        | 1.05     |
| time/                 |          |
|    fps                | 841      |
|    iterations         | 100      |
|    time_elapsed       | 11       |
|    total_timesteps    | 10000    |
| train/                |          |
|    entropy_loss       | -1.54    |
|    explained_variance | 0.00327  |
|    learning_rate      | 0.0001   |
|    n_updates          | 99       |
|    policy_loss        | 0.795    |
|    value_loss         | 0.999    |
------------------------------------
Шаг времени: 11000, Средняя награда: 0.042
Шаг времени: 12000, Средняя награда: 0.056
Шаг времени: 13000, Средняя награда: 0.061
Шаг времени: 14000, Средняя награда: 0.04
Шаг времени: 15000, Средняя награда: 0.064
Шаг времени: 16000, Средняя награда: 0.045
Шаг времени: 17000, Средняя награда: 0.075
Шаг

## Сохраняем

In [15]:
!zip -r /content/all_files.zip /content

updating: content/ (stored 0%)
updating: content/.config/ (stored 0%)
updating: content/.config/gce (stored 0%)
updating: content/.config/active_config (stored 0%)
updating: content/.config/default_configs.db (deflated 98%)
updating: content/.config/.last_update_check.json (deflated 22%)
updating: content/.config/logs/ (stored 0%)
updating: content/.config/logs/2024.07.31/ (stored 0%)
updating: content/.config/logs/2024.07.31/19.24.37.450020.log (deflated 85%)
updating: content/.config/logs/2024.07.31/19.24.38.677192.log (deflated 58%)
updating: content/.config/logs/2024.07.31/19.24.27.578537.log (deflated 58%)
updating: content/.config/logs/2024.07.31/19.24.48.277971.log (deflated 57%)
updating: content/.config/logs/2024.07.31/19.24.48.872630.log (deflated 56%)
updating: content/.config/logs/2024.07.31/19.24.05.521263.log (deflated 93%)
updating: content/.config/.last_survey_prompt.yaml (stored 0%)
updating: content/.config/.last_opt_in_prompt.yaml (stored 0%)
updating: content/.confi

In [17]:
from google.colab import files
files.download('/content/all_files.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>